In [15]:
import warnings

warnings.filterwarnings("ignore")
import os
import glob
from datetime import datetime
import io
import pandas as pd
import numpy as np

import statsmodels.api as sm
import logging

# 로그 설정
logging.basicConfig(filename='error.log', level=logging.ERROR)

from itertools import product
from tqdm import tqdm

# data_h_df = pd.read_csv('./temp_data.csv', index_col='dt', parse_dates=['dt'])


def sarima_learn(data_h_df):
    p, q = range(0, 2), range(0, 2)
    d = range(0, 1)
    P, Q = range(0, 2), range(0, 2)
    D = range(0, 1)
    m = 24
    trend_pdq = list(product(p, d, q))
    seasonal_pdq = [
        (candi[0], candi[1], candi[2], m) for candi in list(product(P, D, Q))
    ]

    # all_list = data_h_df.TABLE_INDEX.unique()
    # ld_nm_list=all_list[:]
    # ld_nm_list=all_list[5:15]

    file_list = []

    # print('부하명 : ', ld_nm)
    ## 최소학습
    # train_df = data_h_df['DIS'][len(data_h_df[data_h_df['TABLE_INDEX'] == ld_nm]) - (24*60):len(data_h_df[data_h_df['TABLE_INDEX'] == ld_nm]) - (24*30)]
    # test_df = data_h_df['DIS'][len(data_h_df[data_h_df['TABLE_INDEX'] == ld_nm]) - (24*30):]
    cut_idx = round(len(data_h_df["val"]) * 0.8)

    train_df = data_h_df["val"][:cut_idx]
    test_df = data_h_df["val"][cut_idx:]
    # print(train_df)

    train_df = train_df.asfreq("H", method="ffill")
    test_df = test_df.asfreq("H", method="ffill")
    Y_train_feR = train_df.copy()
    Y_test_feR = test_df.copy()
    Y_train_feR = Y_train_feR + 0.00000000000000000001
    Y_test_feR = Y_test_feR + 0.00000000000000000001

    try:
        ##  AUTO SARIMAX (최적파라미터)
        AIC = []
        print("11111111111111")
        SARIMAX_order = []
        for trend_param in tqdm(trend_pdq):
            for seasonal_params in seasonal_pdq:
                try:
                    result = sm.tsa.SARIMAX(
                        Y_train_feR,
                        trend="c",
                        order=trend_param,
                        seasonal_order=seasonal_params,
                    ).fit()
                    print(
                        "Fit SARIMAX: trend_order={} seasonal_order={} AIC={}, BIC={}".format(
                            trend_param,
                            seasonal_params,
                            result.aic,
                            result.bic,
                            end="\r",
                        )
                    )
                    AIC.append(result.aic)
                    SARIMAX_order.append([trend_param, seasonal_params])
                except Exception as e:
                    # 모든 예외에 대한 처리
                    logging.error("예외 발생: %s", str(e))
                    continue
        ## Parameter Selection
        print(
            "The smallest AIC is {} for model SARIMAX{}x{}".format(
                min(AIC),
                SARIMAX_order[AIC.index(min(AIC))][0],
                SARIMAX_order[AIC.index(min(AIC))][1],
            )
        )
        print("222222222222222")
        fit_ts_sarimax = sm.tsa.SARIMAX(
            Y_train_feR,
            trend="c",
            order=SARIMAX_order[AIC.index(min(AIC))][0],
            seasonal_order=SARIMAX_order[AIC.index(min(AIC))][1],
        ).fit()

        ## Prediction
        print("33333333333333")
        pred_tr_ts_sarimax = fit_ts_sarimax.predict()
        pred_te_ts_sarimax = fit_ts_sarimax.get_forecast(
            steps=len(Y_test_feR)
        ).predicted_mean
        pred_te_ts_sarimax_ci = fit_ts_sarimax.get_forecast(
            steps=len(Y_test_feR)
        ).conf_int()

        def forecast_one_step():
            pred = fit_ts_sarimax.get_forecast(1)
            return pred.predicted_mean, pred.conf_int()
        print("4444444444444444")
        result_df = pd.DataFrame()
        res = pd.DataFrame()
        for values in Y_test_feR.val.values:
            pred, ci = forecast_one_step()
            result_df = pd.concat([pd.DataFrame({"predict": pred}), ci], axis=1)
            res = pd.concat([res, result_df])
            fit_ts_sarimax = fit_ts_sarimax.extend(np.array([values]))
        print("5555555555555555")
        val_df = pd.concat([pd.DataFrame(res.predict), Y_test_feR], axis=1)
        val_df["dif"] = (val_df.predict - val_df.val).abs()
        val_df["dif/val"] = (val_df["dif"] / val_df["val"] * 100).abs()
        print("666666666666666")
        file_list.append(
            [
                "test",
                SARIMAX_order[AIC.index(min(AIC))][0],
                SARIMAX_order[AIC.index(min(AIC))][1],
                100 - val_df["dif/val"].mean(),
            ]
        )
        fit_ts_sarimax.save("./ld_sarima_" + ".pkl")
        print("7777777777777")
        one_list = []
        one_list.append(
            [
                "test",
                SARIMAX_order[AIC.index(min(AIC))][0],
                SARIMAX_order[AIC.index(min(AIC))][1],
                100 - val_df["dif/val"].mean(),
            ]
        )
        with open("./parameter.csv", "a") as f:
            for param in one_list:
                print(str(param))
                f.writelines("%s\n" % str(param))
        one_list = []

    except Exception as e:
        # 모든 예외에 대한 처리
        logging.error("예외 발생: %s", str(e))
        file_list.append(["test", "예외발생"])


# sarima_learn(data_h_df)


In [16]:
import pandas as pd

df = pd.read_csv("./script/temp_data2.csv", index_col="dt", parse_dates=["dt"])
sarima_learn(df)

11111111111111


  0%|          | 0/4 [00:00<?, ?it/s]

Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(0, 0, 0, 24) AIC=242.26958585702675, BIC=245.54475817647952
Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(0, 0, 1, 24) AIC=229.80346947874, BIC=234.71622795791916
Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(1, 0, 0, 24) AIC=218.62543448881746, BIC=223.53819296799662


 25%|██▌       | 1/4 [00:00<00:02,  1.42it/s]

Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(1, 0, 1, 24) AIC=220.62543457377205, BIC=227.17577921267758
Fit SARIMAX: trend_order=(0, 0, 1) seasonal_order=(0, 0, 0, 24) AIC=215.6437978032091, BIC=220.55655628238827
Fit SARIMAX: trend_order=(0, 0, 1) seasonal_order=(0, 0, 1, 24) AIC=206.3472769874057, BIC=212.89762162631126
Fit SARIMAX: trend_order=(0, 0, 1) seasonal_order=(1, 0, 0, 24) AIC=202.41524374036425, BIC=208.96558837926978


...

Fit SARIMAX: trend_order=(0, 0, 1) seasonal_order=(1, 0, 1, 24) AIC=204.415243927091, BIC=212.60317472572294
Fit SARIMAX: trend_order=(1, 0, 0) seasonal_order=(0, 0, 0, 24) AIC=190.6120612794692, BIC=195.52481975864836
Fit SARIMAX: trend_order=(1, 0, 0) seasonal_order=(0, 0, 1, 24) AIC=186.0247795705439, BIC=192.57512420944943
Fit SARIMAX: trend_order=(1, 0, 0) seasonal_order=(1, 0, 0, 24) AIC=185.85837840222595, BIC=192.4087230411315


 75%|███████▌  | 3/4 [00:02<00:00,  1.25it/s]

Fit SARIMAX: trend_order=(1, 0, 0) seasonal_order=(1, 0, 1, 24) AIC=187.8523005579916, BIC=196.04023135662354
Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(0, 0, 0, 24) AIC=192.54329085201078, BIC=199.0936354909163
Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(0, 0, 1, 24) AIC=188.01925524094108, BIC=196.207186039573
Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(1, 0, 0, 24) AIC=187.8556915987865, BIC=196.04362239741843


100%|██████████| 4/4 [00:03<00:00,  1.13it/s]
ERROR:root:예외 발생: 'Series' object has no attribute 'val'


Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(1, 0, 1, 24) AIC=189.85270761388276, BIC=199.67822457224108
The smallest AIC is 185.85837840222595 for model SARIMAX(1, 0, 0)x(1, 0, 0, 24)
222222222222222
33333333333333
4444444444444444


In [11]:
import threading

def list_threads():
    for thread in threading.enumerate():
        print(f"Thread name: {thread.name}")

if __name__ == "__main__":
    list_threads()

Thread name: MainThread
Thread name: IOPub
Thread name: Heartbeat
Thread name: Control
Thread name: IPythonHistorySavingThread
Thread name: Thread-4


In [54]:
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import font_manager, rc
import seaborn as sns

# 예측값(pred_value)과 실제값(real_value) 예시 데이터
dates = [datetime(2023, 9, 12, 8), datetime(2023, 9, 12, 9), datetime(2023, 9, 12, 10), datetime(2023, 9, 12, 11), datetime(2023, 9, 12, 12),  ]
pred_value = [10, 15, 12, 18, 20]
real_value = [8, 14, 10, 16, 19]

def upload_to_today(filename):
    today = datetime.now().strftime("%Y-%m-%d")
    return f"media/images/{today}/{filename[:-4]}"

folder_name = upload_to_today('data9.csv')
# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [56]:
# 산점도 그래프
def savefig_scatter():
    sns.set(style='whitegrid')
    plt.figure(figsize=(10, 6))

    sns.scatterplot(x=dates, y=pred_value, label='Prediction', color='blue', marker='o')
    sns.scatterplot(x=dates, y=real_value, label='Actual', color='red', marker='x')

    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('Scatter Plot of Prediction vs. Actual')

    plt.legend()
    plt.xticks(rotation=45)  # Rotate x-axis labels

    plt.savefig(folder_name + '/scatter.png')
    plt.close()

# 히스토그램 그래프
def savefig_histogram():
    plt.figure(figsize=(10, 6))

    sns.histplot(pred_value, label='Prediction', color='blue', kde=True)
    sns.histplot(real_value, label='Actual', color='red', kde=True)

    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Prediction vs. Actual')

    plt.legend()

    plt.savefig(folder_name + '/histogram.png')
    plt.close()

# 박스 플롯 그래프
def savefig_boxplot():
    plt.figure(figsize=(10, 6))

    sns.boxplot(data=[pred_value, real_value], palette=['blue', 'red'])
    
    plt.ylabel('Value')
    plt.title('Box Plot of Prediction vs. Actual')

    plt.xticks([0, 1], ['Prediction', 'Actual'])
    plt.savefig(folder_name + '/boxplot.png')
    plt.close()

# 바이올린 플롯 그래프
def savefig_violinplot():
    plt.figure(figsize=(10, 6))

    sns.violinplot(data=[pred_value, real_value], palette=['blue', 'red'])
    
    plt.ylabel('Value')
    plt.title('Violin Plot of Prediction vs. Actual')

    plt.xticks([0, 1], ['Prediction', 'Actual'])

    plt.savefig(folder_name + '/violinplot.png')
    plt.close()

# 각 함수를 호출하여 그래프를 생성하고 저장
savefig_scatter()
savefig_histogram()
savefig_boxplot()
savefig_violinplot()